In [81]:
#importing libraries
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from sklearn.metrics.pairwise import cosine_similarity
from typing import List,Dict,Any,Tuple
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import os
import numpy as np
import uuid
import chromadb

In [ ]:
#Document Object
doc=Document(
    page_content="this is the main context I am using to create RAG",
    metadata={
        "source":"example.txt",
        "pages":1,
        "author":"Krish Naik",
        "date_created":"2025-11-05"
    }
)

In [83]:
#Creating a txt file
os.makedirs("../data/text_files",exist_ok=True)

In [84]:
sample_texts=sample_texts={
    "../data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "../data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """
}

for filepath,content in sample_texts.items():
    with open(filepath,"w",encoding="utf-8") as f:
        f.write(content)
print("Sample files created")

Sample files created


In [85]:
#Textloader
#Structure raw text into Langchain Document format
loader=TextLoader("../data/text_files/python_intro.txt",encoding="utf-8")
document=loader.load()

In [86]:
#Directory Loader
#Load all text files from the directory 
dir_loader=DirectoryLoader(
    "../data/text_files",
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding":"utf-8"},
    show_progress=True
)
documents=dir_loader.load()

100%|██████████| 2/2 [00:00<00:00, 3842.70it/s]


In [87]:
dir_loader=DirectoryLoader(
    "../data/pdf",
    glob="**/*.pdf",
    loader_cls=PyMuPDFLoader,
    show_progress=True
)
pdf_documents=dir_loader.load()

100%|██████████| 1/1 [00:00<00:00,  7.96it/s]


In [ ]:
#Embeddings
class EmbeddingManager:
    def __init__(self,model_name:str="all-MiniLM-L6-v2"):
        self.model_name=model_name
        self.model=None
        self._load_model()

    def _load_model(self):
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model=SentenceTransformer(self.model_name)
            print(f"Model loaded sucessfully.")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise
    
    def generate_embeddings(self,texts: List[str]) -> np.ndarray:
        if not self.model:
            raise ValueError("Model not loaded")
        print(f"Generating embeddings for {len(texts)} texts....")
        embeddings=self.model.encode(texts,show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings
    

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded sucessfully.


In [ ]:
#VectoreStore
class VectorStore:
    def __init__(self,collection_name: str="pdf_documents",persist_directory:str="../data/vector_store"):
        self.collection_name=collection_name
        self.persist_directory=persist_directory
        self.client=None
        self.collection=None
        self._initialize_store()